# Prompt best practices

Prompting principles to write effective prompts for LLMs.

## Setup

In [ ]:
# !pip install openai
# !pip install python-dotenv
# !pip install panel

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
DEFAULT_MODEL = "gpt-3.5-turbo"
SYSTEM_ROLE = "system"
USER_ROLE = "user"
ASSISTANT_ROLE = "assistant"

## Utils

We will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/gpt/chat-completions-api).

In [4]:
def get_completion(prompt, model=DEFAULT_MODEL, temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature, # degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

#### Use delimiters to clearly indicate distinct parts of the input
- To avoid a prompt injection
- Delimiters can be anything like: ``` (triple backticks), """ (triple quotes), --- (triple dahses), < > (andle brackets), `<tag> </tag>` (XML tags), ...

*Prompt injection examples*

![Prompt delimiters example](prompt-delimiters.png)

![Prompt injection example](prompt-injection-1.PNG)

![Prompt injection solved](prompt-injection-solved.PNG)

In [6]:
text = f"""
Please explain how Trasformer Architecture work
"""
prompt = f"""
Please proofread and correct the text delimited by triple backticks and rewrite the corrected version. \
Do not write any explanation.
```{text}```
"""

response = get_completion(prompt)
print(response)

Please explain how Transformer Architecture works.


### Ask for a structured output (JSON, HTML,...)

In [ ]:
prompt = f"""
Generate a list of three made-up book titles along \
with their authors and genres.
Provide them in JSON format with the following keys:
book_id, title, author, genre
"""
response = get_completion(prompt)

In [8]:
print(response)

{
  "books": [
    {
      "book_id": 1,
      "title": "The Enigma of Elysium",
      "author": "Aria Nightshade",
      "genre": "Fantasy"
    },
    {
      "book_id": 2,
      "title": "Whispers in the Shadows",
      "author": "Evelyn Blackwood",
      "genre": "Mystery"
    },
    {
      "book_id": 3,
      "title": "Beyond the Veil",
      "author": "Lucas Stormrider",
      "genre": "Science Fiction"
    }
  ]
}


### Ask the model to check whether conditions are sactisfied

Example 1: Make a cup of coffee. Add `stop shot` 

In [ ]:
text_1 = f"""
Making a cup of tea is easy! \ 
First, you need to get some water boiling. \ 
While that's happening, grab a cup and put a tea bag in it. \ 
Once the water is hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a few minutes, \ 
take out the tea bag. If you like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

In [ ]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

![Prompt escape example](prompt-escaped.PNG)

In [ ]:
prompt = f"""
Which color is a pineapple? Choose from the options below: \
a) blue
b) pink
c) white
"""
response = get_completion(prompt)
print(response) # d) yellow

In [ ]:
prompt = f"""
Which color is a pineapple? Choose from the options below: \
a) blue
b) pink
c) white
d) none of the above
"""
response = get_completion(prompt)
print(response) 

### Few-shot prompting

Give successful examples of completing tasks.
Then ask the model to perform the task

In [ ]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
Bạn là nhà văn sáng tác văn theo một văn phong thống nhất.

<độc giả>: Sáng tác về sông Hương.

<nhà văn>: Trong những dòng sông đẹp ở các nước mà tôi thường nghe nói đến, \ 
hình như chỉ sông Hương là thuộc về một thành phố duy nhất. \ 
Trước khi về đến vùng châu thổ êm đềm, nó đã là một bản trường ca của rừng già, rầm rộ \ 
giữa bóng cây đại ngàn, mãnh liệt qua những ghềnh thác, cuộn xoáy như cơn lốc vào \ 
những đáy vực bí ẩn, và cũng có lúc nó trở nên dịu dàng và say đắm giữa những dặm dài \ 
chói lọi màu đỏ của hoa đỗ quyên rừng. Giữa lòng Trường Sơn, sông Hương đã sống \ 
một nửa cuộc đời của mình như một cô gái Digan phóng khoáng và man dại. \ 
Rừng già đã hun đúc cho nó một bản lĩnh gan dạ, một tâm hồn tự do và trong sáng.\ 
Nhưng chính rừng già nơi đây, với cấu trúc đặc biệt có thể lý giải được về mặt khoa học, \ 
đã chế ngự sức mạnh bản năng ở người con gái của mình để khi ra khỏi rừng,\ 
sông Hương nhanh chóng mang một sắc đẹp dịu dàng và trí tuệ,\ 
trở thành người mẹ phù sa của một vùng văn hóa xứ sở. \ 
Nếu chỉ mải mê nhìn ngắm khuôn mặt kinh thành của nó, tôi nghĩ rằng người ta sẽ \ 
không hiểu một cách đầy đủ bản chất của sông Hương với cuộc hành trình gian \ 
truân mà nó đã vượt qua, không hiểu thấu phần tâm hồn sâu thẳm của nó mà dòng \ 
sông hình như không muốn bộc lộ, đã đóng kín lại ở cửa rừng và ném chìa khóa \ 
trong những hang đá dưới chân núi Kim Phụng.

<độc giả>: Sáng tác về sông Sài Gòn.
"""
response = get_completion(prompt)
print(response)

### Specify the steps required to complete a (complex) task

For tasks that are complex to do in a short amount of time, the model may make up a guess.

-> Instruct the model to think longer about a problem

In [ ]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Output a json object that contains the following \
keys: french_summary, num_names where french_summary is \ 
a summary in French the following text delimited by triple \ 
backticks with 1 sentence and the num_names is a number of \ 
name in French summary.


Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Improved approach

In [ ]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Format the output

In [ ]:
prompt_2 = f"""
Your task is to perform the following actions:
1 - Summarizize the following text delimited by <> with one sentence.
2 - Translate the summay into French
3 - List each name in the French summary.
4 - Output a json object that contains the following keys: french_summary, num_names.

Use the following format:
Text: <text to summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)

### Instruct the model to work out its own solution before rushing to a conclusion

In [ ]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

Fix the prompt

In [5]:
prompt = f"""
Your task is to determine if the student's solution \ 
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem.
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution in correct or not.
Don't decide if the student's solution is correct until you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as the actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

To calculate the total cost for the first year of operations, we need to add up the costs of land, solar panels, and maintenance.

Let x be the size of the installation in square feet.

1. Land cost: $100 / square foot
The cost of land is 100 * x = 100x.

2. Solar panel cost: $250 / square foot
The cost of solar panels is 250 * x = 250x.

3. Maintenance cost: $100,000 + $10 / square foot
The maintenance cost is a flat $100,000 per year, plus $10 per square foot. So the maintenance cost is 100,000 + 10 * x = 100,000 + 10x.

Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000.

Is the student's solution the same as the actual solution just calculated:
No.

Student grade:
Incorrect.
